# Custom EnergyPlus Evaluator functions
This notebook shows how to write a custom function that gets applied to the EnergyPlus outputs.

In [1]:
import pandas as pd
import numpy as np

from besos.evaluator import EvaluatorEP
from besos.problem import EPProblem
from besos import eppy_funcs as ef
from besos import eplus_funcs as epf

from besos.parameters import RangeParameter, CategoryParameter, Parameter, FieldSelector
from besos.objectives import MeterReader, VariableReader, time_series_values, EPReader

### Set up an EnergyPlus Evaluator
Here is a standard EnergyPlus Evaluator for editing the lighting power density for the default building and getting the electricity demand time series.  
By default, we get the sum of the variable we specify.

In [2]:
building = ef.get_building()
EPparameters = [Parameter(FieldSelector('Lights', '*', 'Watts per Zone Floor Area'),
                value_descriptor=RangeParameter(8, 12),
                name='Lights Watts/Area')]
EPobjectives = MeterReader('Electricity:Facility')
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

/home/user/.local/lib/python3.7/site-packages/besos/parameters.py:425: FutureWarning: Use value_descriptors instead of value_descriptor.
  FutureWarning("Use value_descriptors instead of value_descriptor.")


(1672453961.7713666,)

### Time series data

Sometimes we want to get back an entire time series, for example for linking to the Energy Hub model (see [here](../EnergyHub/EPtoEHEvaluators.ipynb)).  
For that we can specify `func=time_series_values`, which comes as part of besos.objectives.

In [3]:
EPobjectives = MeterReader('Electricity:Facility',func=time_series_values)
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result[0]

/home/user/.local/lib/python3.7/site-packages/besos/objectives.py:218: UserWarning: `time_series_values` is incomplete, and returns raw values, not time series values.
  "`time_series_values` is incomplete, and returns raw values, not time series values."


0     5.041708e+07
1     5.142561e+07
2     5.193728e+07
3     5.111495e+07
4     4.728000e+07
5     4.371469e+07
6     5.956208e+07
7     5.870644e+07
8     6.031615e+07
9     6.127009e+07
10    6.202866e+07
11    6.294521e+07
12    6.369996e+07
13    6.431286e+07
14    6.507253e+07
15    6.522390e+07
16    6.471212e+07
17    6.365803e+07
18    6.167204e+07
19    6.648230e+07
20    6.682431e+07
21    6.601229e+07
22    4.320751e+07
23    4.910526e+07
24    1.431370e+07
25    1.310912e+07
26    1.431370e+07
27    1.310912e+07
28    1.431370e+07
29    1.310912e+07
30    1.551827e+07
31    8.609274e+06
32    7.227474e+06
33    7.227474e+06
34    7.227474e+06
35    7.227474e+06
36    7.227474e+06
37    7.227474e+06
38    7.227474e+06
39    7.227474e+06
40    9.991074e+06
41    1.551827e+07
42    1.407278e+07
43    1.310912e+07
44    1.431370e+07
45    1.310912e+07
46    1.431370e+07
47    1.310912e+07
Name: Value, dtype: float64

### Define a custom function
Now we will define the function that we will apply to the EnergyPlus output data.  
The function `highloadhoursfunc` counts the hours in which the load is above a threshold.

In [4]:
def highloadhoursfunc(result):
    threshold = 1e7
    highloadhours = sum(result.data['Value'] >= threshold)
    return highloadhours

By specifying `func=highloadhoursfunc`, the result is the number of hours in which `Electricity:Facility` is above the threshold.

In [5]:
EPobjectives = MeterReader('Electricity:Facility',func=highloadhoursfunc)
problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

(38,)

## Find all availble output values
below is a code you can use to find all availble output values.

In [6]:
epf.print_available_outputs(building)

['Environment,Site Outdoor Air Drybulb Temperature', 'Hourly']
['Environment,Site Outdoor Air Humidity Ratio', 'Hourly']
['Environment,Site Outdoor Air Relative Humidity', 'Hourly']
['ATTIC,Zone Mean Air Temperature', 'Hourly']
['CORE_ZN,Zone Mean Air Temperature', 'Hourly']
['PERIMETER_ZN_1,Zone Mean Air Temperature', 'Hourly']
['PERIMETER_ZN_2,Zone Mean Air Temperature', 'Hourly']
['PERIMETER_ZN_3,Zone Mean Air Temperature', 'Hourly']
['PERIMETER_ZN_4,Zone Mean Air Temperature', 'Hourly']
['PSZ-AC:1,Air System Simulation Cycle On Off Status', 'Hourly']
['PSZ-AC:2,Air System Simulation Cycle On Off Status', 'Hourly']
['PSZ-AC:3,Air System Simulation Cycle On Off Status', 'Hourly']
['PSZ-AC:4,Air System Simulation Cycle On Off Status', 'Hourly']
['PSZ-AC:5,Air System Simulation Cycle On Off Status', 'Hourly']
['PSZ-AC:1,Air System Outdoor Air Economizer Status', 'Hourly']
['PSZ-AC:1,Air System Outdoor Air Flow Fraction', 'Hourly']
['PSZ-AC:2,Air System Outdoor Air Economizer Status', '

You can see there a bunch of different values that you can use that have the format 'key_value,variable_name'. These are variables and not meters. You can call on these outputs using the VariableReader. 

In [7]:
EPobjectives = VariableReader(key_value='PSZ-AC:5',variable_name='Air System Total Heating Energy',frequency='Hourly',func=time_series_values)

problem = EPProblem(EPparameters,EPobjectives)
evaluator = EvaluatorEP(problem,building)
result = evaluator([8])
result

(0     9.863781e+05
 1     9.105029e+05
 2     1.138129e+06
 3     9.105029e+05
 4     1.138129e+06
 5     9.105029e+05
 6     1.365754e+06
 7     1.365754e+06
 8     1.365754e+06
 9     1.365754e+06
 10    1.365754e+06
 11    1.365754e+06
 12    1.365754e+06
 13    1.365754e+06
 14    1.365754e+06
 15    1.365754e+06
 16    1.365754e+06
 17    1.365754e+06
 18    1.365754e+06
 19    1.365754e+06
 20    1.365754e+06
 21    1.365754e+06
 22    2.276257e+05
 23    1.138129e+06
 24    1.365248e+07
 25    1.048507e+07
 26    1.371594e+07
 27    1.052901e+07
 28    1.376205e+07
 29    1.056124e+07
 30    1.254686e+07
 31    1.754508e+07
 32    1.728680e+07
 33    1.716945e+07
 34    1.709543e+07
 35    1.703944e+07
 36    1.699434e+07
 37    1.695709e+07
 38    1.692752e+07
 39    1.690228e+07
 40    1.687979e+07
 41    1.686032e+07
 42    1.152584e+07
 43    1.005320e+07
 44    1.338084e+07
 45    1.032418e+07
 46    1.355788e+07
 47    1.042188e+07
 Name: Value, dtype: float64,)